In [1]:
import numpy as np
import pandas as pd 

## Read files

In [3]:
url = 'C:/Users/User/talkingdata/'
app_events = pd.read_csv(url + 'app_events.csv')
app_labels = pd.read_csv(url + 'app_labels.csv')
label_categories = pd.read_csv(url + 'label_categories.csv')

In [ ]:
print(app_events.head(2),'\n')
print(app_labels.head(2),'\n')
print(events.head(2),'\n')
print(label_categories.head(2),'\n')

## Clean table app_labels.csv

### Drop duplicated data

In [ ]:
print('sum of duplicated data: ',app_labels.duplicated().sum())
app_labels.drop_duplicates(inplace = True)
print('shape: ',app_labels.shape)
app_labels.head()

### Check missing data

In [ ]:
print('missing data: ',app_labels.isnull().any().sum())

## Clean table label_categories.csv

### Check duplicated data

In [ ]:
print('duplicated data: ',label_categories.duplicated().sum())
print('shape: ',label_categories.shape)
label_categories.head()

### Drop missing data

In [ ]:
print('missing data:\n',label_categories.isnull().sum())
label_categories.dropna(inplace = True)
print('\nlabel_categories.shape: ',label_categories.shape)

## Merge app_labels and label_categories 

In [ ]:
app_categories_mer = pd.merge(app_labels,label_categories,how = 'inner',on = 'label_id')
app_categories_mer.drop(columns='label_id',inplace=True)
print('app_categories_mer.shape: ',app_categories_mer.shape,'\n')
app_categories_mer.head()

### one hot encoding

In [ ]:
app_categories_mer = pd.get_dummies(app_categories_mer)
print('app_categories_mer.shape: ',app_categories_mer.shape)
app_categories_mer.head()

### groupby app_id

In [ ]:
app_categories_mer = app_categories_mer.groupby(['app_id']).sum()
print('app_categories_mer.shape',app_categories_mer.shape)
app_categories_mer.head()

In [ ]:
import gc
del app_labels
del label_categories
print(gc.collect())

## Clean table app_events.csv

In [ ]:
print('app_events.shape: ',app_events.shape)
app_events.head()

### Drop is_installed and is_active, filer by is_active == 1

In [ ]:
app_events = app_events[app_events['is_active']==1]
app_events.drop(['is_installed','is_active'],inplace = True,axis = 1)
print('app_events.shape: ',app_events.shape)
app_events.head()

### Merge events

In [ ]:
app_categories_mer.reset_index(inplace = True)
df_events = pd.merge(app_events,app_categories_mer,how = 'inner',on = 'app_id')
print('df_events.shape: ',df_events.shape)
df_events.head()

In [ ]:
import gc
del app_events
del app_categories_mer
print(gc.collect())

### drop app_id

In [ ]:
df_events.drop(columns = ['app_id'],inplace = True)
print('df_events.shape: ',df_events.shape)
df_events.head()

### write to csv

In [ ]:
df_events.to_csv(url + 'model/df_events.csv')

### Group by event_id

#### sort by event_id

In [ ]:
df_events = df_events.sort_values(['event_id'])
df_events.head()

#### Define Functions split-groupby-concat
processing df_events.iloc[0:12700000, :]

In [ ]:
# split table and to_csv
def split_save(loops):
    for i in range(loops):
        temp = df_events.iloc[i*100000 : (i+1)*100000, :]
        name = 'eventGroupby/df_events'+str(i+1)
        temp.to_csv(url+name+'.csv')
    return 'split finished'

# group by events_id and to_csv
def groupby_events_id(loops):
    for i in range(loops):
        name = 'eventGroupby/df_events_groupby'+str(i+1)
        dataframe = pd.read_csv(url + 'eventGroupby/df_events' + str(i+1) + '.csv',index_col='event_id')
        dataframe.drop(columns=['Unnamed: 0'],inplace=True)
        dataframe = dataframe.groupby(['event_id']).sum()
        dataframe.to_csv(url + name + '.csv')
    return 'groupby finished'

# concat table and to_csv
def concat_table(loops):
    L=[]
    for i in range(loops):
        L.append(pd.read_csv(url + 'eventGroupby/df_events_groupby' + str(i+1) + '.csv'))
    dataframe = pd.concat(L,axis=0)
    dataframe.to_csv(url + 'eventGroupby/df_events_concat.csv')
    return 'concat table finished'

In [ ]:
import gc
print(gc.collect())

#### Call functions
processing df_events.iloc[0:12700000, : ]

In [ ]:
split_save(127)
groupby_events_id(127)
concat_table(127)

#### Read groupby and write to_csv
processing df_events.iloc[12700000:12732996, : ]

In [ ]:
df_events_last = df_events.iloc[12700000 : 12732997,:]
df_events_last = df_events_last.groupby('event_id').sum()
df_events_last.to_csv(url+'eventGroupby/df_events_last.csv')
df_events_last.shape

#### Read all split table

In [ ]:
df_events_last = pd.read_csv(url+'eventGroupby/df_events_last.csv',index_col=None)
df_events_concat = pd.read_csv(url+'eventGroupby/df_events_concat.csv',index_col=None)
df_events_concat = df_events_concat.iloc[:,1:]
print('df_events_concat.shape: ',df_events_concat.shape)
print('df_events_last.shape: ',df_events_last.shape)

#### concat  all slpit table

In [ ]:
df_events_groupby = pd.concat((df_events_concat,df_events_last),axis=0)
df_events_groupby.shape

In [ ]:
import gc
del df_events_concat
del df_events_last
print(gc.collect())

#### groupby all split table

In [ ]:
# check duplicated data
df_events_groupby.duplicated(subset='event_id').sum() # if sum != 0, groupby again

In [ ]:
df_events_groupby1 = df_events_groupby.iloc[:500000]
df_events_groupby2 = df_events_groupby.iloc[500000:1000000]
df_events_groupby3 = df_events_groupby.iloc[1000000:]

df_events_groupby1 = df_events_groupby1.groupby('event_id').sum()
df_events_groupby2 = df_events_groupby2.groupby('event_id').sum()
df_events_groupby3 = df_events_groupby3.groupby('event_id').sum()

df_events_groupby=pd.concat([df_events_groupby1,df_events_groupby2,df_events_groupby3],axis=0)
df_events_groupby.head()

#### write to_csv

In [ ]:
df_events_groupby.to_csv(url+'eventGroupby/df_events_groupby.csv')

## events

### Readfiles

In [ ]:
# df_events_groupby = pd.read_csv(url+'eventGroupby/df_events_groupby.csv')
events = pd.read_csv(url + 'events.csv')
print('events.shape: ',events.shape)
events.head()

### drop time and location

In [ ]:
events.drop(['timestamp', 'longitude', 'latitude'], inplace = True, axis = 1)
events.head()

### Merge events on event_id

In [ ]:
events = pd.merge(events,df_events_groupby, on = 'event_id', how = 'inner')
print('events.shape: ',events.shape)
events.head()

In [ ]:
import gc
del df_events_groupby
print(gc.collect())

### Drop event_id

In [4]:
events=pd.read_csv(url + 'model/app_matrix.csv',index_col=None)
events.drop(['Unnamed: 0','event_id'],inplace = True, axis = 1)
events.head()

,device_id,category_1 free,category_1 reputation,category_1 vitality,category_3 kindom game,category_80s Japanese comic,category_90s Japanese comic,category_A beauty care,category_A shares,category_ARPG,...,category_travel,category_tribe,category_trickery,category_unknown,category_video,category_violence comic,category_vitality,category_war chess,category_weibo,category_zombies game
0,-6401643145415154744,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1476664663289716375,1,1,0,0,0,0,0,0,0,...,0,0,0,7,1,0,0,0,0,0
2,5990807147117726237,0,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,1,0
3,-2073340001552902943,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9070651185984875886,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group by device_id

In [6]:
events = events.groupby(['device_id']).sum()
print('events.shape: ',events.shape)
events.head()

events.shape:  (60646, 473)


,category_1 free,category_1 reputation,category_1 vitality,category_3 kindom game,category_80s Japanese comic,category_90s Japanese comic,category_A beauty care,category_A shares,category_ARPG,category_Academic Information,...,category_travel,category_tribe,category_trickery,category_unknown,category_video,category_violence comic,category_vitality,category_war chess,category_weibo,category_zombies game
device_id,,,,,,,,,,,,,,,,,,,,,
-9222956879900151005,0,16,0,0,0,0,0,0,0,0,...,0,0,0,244,0,0,0,0,0,0
-9222661944218806987,7,0,0,0,0,0,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0
-9222399302879214035,0,0,0,0,0,0,0,0,0,0,...,0,0,0,40,0,0,0,0,0,0
-9221825537663503111,28,0,0,0,0,0,0,0,0,0,...,0,0,0,20,45,0,0,0,0,0
-9221767098072603291,5,6,0,0,0,0,0,0,0,0,...,0,0,0,46,2,0,0,0,0,0


### Transfer non-zero numbers to one 

In [10]:
events = events.applymap(lambda x:1 if x!=0 else 0)
events.head()

,category_1 free,category_1 reputation,category_1 vitality,category_3 kindom game,category_80s Japanese comic,category_90s Japanese comic,category_A beauty care,category_A shares,category_ARPG,category_Academic Information,...,category_travel,category_tribe,category_trickery,category_unknown,category_video,category_violence comic,category_vitality,category_war chess,category_weibo,category_zombies game
device_id,,,,,,,,,,,,,,,,,,,,,
-9222956879900151005,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
-9222661944218806987,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
-9222399302879214035,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
-9221825537663503111,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
-9221767098072603291,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


### Write to_csv

In [12]:
events.to_csv(url + 'model/category_matrix.csv')